In [283]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from scipy.sparse import csr_matrix
from scipy.sparse import spdiags
import numpy.matlib


two_moon=datasets.make_moons(n_samples=10,noise=0.1)
X3=two_moon[0]
KM3=KMeans(n_clusters=3,init='random',random_state=5)
KM3.fit(X3)
index=KM3.predict(X3)
#plt.scatter(X3[:,0],X3[:,1],c=list(index))

# KMM_Function
## **input:**
- x : Data matrix
- c : The number of cluster
- m : The number of subcluster
- k : The number of neighbor prototype
    
## **output:**
- lakmm : The <font color='red'>cluster</font> assignment for each point
- lamm : The <font color='red'>sub-cluster</font> assignment for each point
- A : Coordinates of the multiple means 

    

In [47]:
import numpy as np

class Clustering(object):
    
    def __init__(self): pass
    
    def k_multiple_means(x,c,m,k=5):
        
        iterTimes=1000             #迭代次數
        n=np.size(x,axis=0)        #共有幾筆資料
        dim=np.size(x,axis=1)      #每筆資料的維度
        
        if m < 6:                  # k 值必須小於等於 5
            k=c-1
    
        
        A = np.random.random([m,dim])    #亂數產生k個dim維度的值 ----------- (initial) The prototype matrix 
            
        #S：connection probability matrix(initial)
        S=np.array(np.ones([n,m]))   
        
        #--------用k-means找出初始index--------
            
        KM=KMeans(n_clusters=m,init='random',random_state=6)
        KM.fit(x)
        A_index=list(KM.predict(x))                            #A_index : 紀錄 Data point 屬於哪個 Prototype
        
        for count_iter in range(iterTimes):               
            
            old_A=np.copy(A)  
            A=meanIdex(x,A_index,m,S)    
             
            #--------判斷收斂--------
            th=np.linalg.norm(A-old_A)                       #計算兩個prototype的距離

            if th<np.spacing(1):                             #電腦區分兩個數的差的最小精度，當兩個數的差小於這個值，則電腦認為兩個數相等
                break
        
        A=meanIdex(x,A_index,m,S)
        #--------做完約束的 mm ，就是 A_index
        
        return A,A_index

# 測試視窗

In [374]:
A=Clustering.k_multiple_means(X3,3,6,1)
k=5
n=np.size(X3,axis=0)     #計算 Data point 數量
m=6     #計算 Prototype 數量

#GSBG
#從 Bigraph 看會比較清楚
z,y,distX,k_index=ConstructA_NP(X3,A[0])      #預設 k=5，(Data Point 鄰近 k 個 Prototype) 頭到尾
zT,yT,disXT,k_indexT=ConstructA_NP(A[0],X3)   #(Prototype 鄰近 k 個 Data Point) 尾到頭

# 給定 lambda 值
l = (y+yT)/2

#求平均機率
z0 = (z+zT.T)/2




#svd2uv
n,m=np.shape(z0)


z0=z0/np.sum(z0,axis=1)                    #正規化機率

Sum1 = np.sum(z0,axis=1)
D1z = spdiags(Sum1.T, 0 , n, n).toarray()

Sum2 = np.sum(z0,axis=0)
D2z = spdiags(Sum2, 0 , m, m).toarray()

#轉置矩陣 （須符合矩陣乘法）
Z1 = D1z * z0 * D2z
LZ=Z1.T*Z1

eig1(LZ,3)


[0.40966832 0.30515144 0.56013827 0.61361919 0.27947506 0.6361556 ]
[0.636155601537884, 0.6136191870756781, 0.5601382728532036, 0.4096683152107441, 0.3051514401688691, 0.279475063904663]

[5 3 2 0 1 4]


---
## Function : meanIdex 

### 功能 : 更新 A (Prototype 座標 ) ----------> 對應公式（5）

#### **(input)**
- **x**       : Data Point<br>
- **A_index** : 紀錄 Data point 屬於哪個 Prototype (Label值)<br>
- **m**       : sub-cluster(prototype)數量<br>
- **S**       : 連接機率矩陣<br>
    
#### **(return)**
- 更新後的矩陣 **A** (Prototype 座標)<br>

In [35]:
def meanIdex(x,A_index,m,S):

    A=np.zeros([m,2])
    n=np.size(x,axis=0)
    
    for j in range(m):
            index=0               #為了符合先乘除後加減，判斷使用
            mu=0
            su=0
            for i in range(n):       
                if A_index[i]==j:
                    
                    if index==0:
                        mu=mu+x[i]
                        mu=mu*S[i][j]
                        index=1
                    else:
                        mu=mu+(x[i]*S[i][j])
                    
                    su=su+S[i][j]
                    
            A[j]=mu/su       
    
    return A

---
## Function : CSBG

### 功能 : 約束 Bigraph

#### **(input)**
- **x**       : Data Point<br>
- **c**       : The number of cluster<br>
- **A**       : Coordinates of the multiple means<br>

    
#### **(return)**
- lakmm : The cluster assignment for each point
- lamm : The sub-cluster assignment for each point
- bigraph : Bipartite graph (Data point and neighbor prototype)
- A : Coordinates of the multiple means 

In [232]:
def CSBG(x,c,A):
    
    n=np.size(x,axis=0)
    m=np.size(A,axis=0)
    
    z,y,distX,k_index=ConstructA_NP(x,A)      #預設 k=5
    print(z)
    

---
## Function : ConstructA_NP 

### 功能 : 找到 Data Point 鄰近的 k 個 Prototype。 

#### **(input)**
- **x**       : Data Point<br>
- **A**       : Prototype 座標矩陣<br>
- **k**       : 鄰近Prototype的數量<br>
- **isSparse**       :  <br>
    
#### **(return)**
- **z** : (稀疏矩陣)儲存鄰近 Prototype 的機率<br>
- **y** : (gamma)算平均值，當鄰近 Prototype 的距離超過平均值則不連接<br>
- **dist** : 計算每個 Data Point 與 每個 Prototype 的距離
- **k_index**

In [269]:
def ConstructA_NP(x,A,k=5,isSparse=1):
    
    # k 不能大於 sub_cluster 的數量
    
    n=np.size(x,axis=0)     #計算 Data point 數量
    m=np.size(A,axis=0)     #計算 Prototype 數量
    dist=np.zeros([n,m])    #初始化距離矩陣
    
    for i in range(m):
        c=A[i]
        dist[:,i] = np.linalg.norm(x-c,axis=1)     #計算每個 Data Point 與 每個 Prototype 的距離
    
        
    disXt = np.copy(dist)                                    #會修改到 dist 的值，先做備份
    
    k_dist = np.zeros([n,k+1])                               #(初始化 0）生成 n 個 Data Point * (k+1)鄰近點矩陣
    k_index = np.zeros([n,k])                                
    tmp = np.array(np.zeros([n,k]))
   
    for i in range(k+1):
    
        k_dist[:,i]=disXt.min(axis=1)
        if i!=k:                                             #對照 id(:,end) = [];
            k_index[:,i]=disXt.argmin(axis=1)
            for j in range(n):
                a = k_index[j][i]
                disXt[j][int(a)]=1000000000000000

    y=0

    sum_matrix = np.sum(k_dist,axis=1)         

    for i in range(n): #計算 k+1 之前的總和
    
        sum_matrix[i]-=k_dist[i][k]


    for i in range(n): #計算 y 值 ，matlab中是以矩陣方式個別存入，所以在計算tmp時可以直接用 y 的值
    
        y=y+((k*k_dist[i][k])-sum_matrix[i])                 

    y=0.5*y/n

    for i in range(n):
    
        for j in range(k):
        
            tmp[i][j]=(k_dist[i][k]-k_dist[i][j])/((k*k_dist[i][k])-sum_matrix[i])      #算出與鄰近prototype連接機率

    #以下處理是為了長度相符-------------------------------
    count=[]

    count=np.matlib.repmat(np.arange(n), 1, k) #產生 k次 1維 (0~n)數值
    k_index=k_index.reshape(1,n*k)
    tmp=tmp.reshape(1,n*k)

    z = csr_matrix((tmp[0], (count[0], k_index[0])),shape=(n, m))
    
    return z,y,dist,k_index

---
## Function : svd2uv 

### 功能 : 奇異值 (svd) 轉 u v 矩陣值 

#### **(input)**
- **z**       : Connection probability matrix<br>
- **c**       : The number of clusters<br>

    
#### **(return)**
- **z(BiGraph)** : 正規化後的稀疏矩陣<br>
- **u** : <br>
- **v** : <br>
- **evc** : <br>
- **D1z** : <br>
- **D2z** : <br>

In [ ]:
def svd2uv(z,c):
    
    n,m = np.shape(z)
    
    

In [373]:
def eig1(LZ,c):
    
    n,m = np.shape(LZ)
    
    if c > n:
        c = np.copy(n)
    
    B = np.eye(n)
    
    LZ = (LZ + LZ.T)/2
    
    v,d = np.linalg.eig(LZ)
    
    d = np.diag(d)
    d = np.abs(d)
    
    #降冪
    idx=np.argsort(-d)          #index
    d = sorted(d,reverse=True)  #value

